In [68]:
from tqdm import tqdm

from PIL import Image

import sys
import os
import glob
import random
import time

import cv2

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from collections import deque, Counter

In [69]:
class EGD_RegionGrow():
    def __init__(self, image: np.array) -> None:
        self.image = image
        self.gray_image = cv2.cvtColor(self.image, cv2.COLOR_BGR2GRAY)
        _, self.binary_image = cv2.threshold(self.gray_image ,25, 255, None)
        self.h, self.w =  self.image.shape[:2]
    
    def apply_region_growing(self):
        # 입력 이미지와 동일한 크기의 결과 이미지를 생성하고 모든 픽셀을 0으로 초기화합니다.
        segmented = np.zeros_like(self.binary_image)
        
        # 방문한 픽셀을 추적하기 위한 방문 여부 배열을 생성합니다.
        visited = np.zeros((self.h, self.w), dtype=np.uint8)
        
        seed = self.get_seed_pixel()
        
        # 영역 색상을 결정하기 위한 시드 픽셀의 색상 값을 가져옵니다.
        seed_color = self.binary_image[seed[0], seed[1]]
        
        # 스택을 사용하여 영역 색칠 작업을 수행합니다.
        stack = []
        stack.append(seed)
               
        while len(stack) > 0:
            # 스택에서 현재 픽셀을 팝합니다.
            current_pixel = stack.pop()
            x, y = current_pixel
            
            # 현재 픽셀을 방문했는지 확인합니다.
            if visited[x, y] == 1:
                continue
            
            # 현재 픽셀과 시드 픽셀 사이의 색상 차이를 계산합니다.
            pixel_color = self.binary_image[x, y]
            color_diff = np.sum(np.abs(pixel_color - seed_color))
            
            # 같은 색상일 경우 현재 픽셀을 영역에 추가합니다.
            if color_diff == 0:
                segmented[x, y] = pixel_color
                visited[x, y] = 1
                
                # 현재 픽셀의 이웃 픽셀을 스택에 추가합니다.
                if x > 0:
                    stack.append((x - 1, y))
                if x < self.w - 1:
                    stack.append((x + 1, y))
                if y > 0:
                    stack.append((x, y - 1))
                if y < self.h - 1:
                    stack.append((x, y + 1))
                    
        return segmented

    def get_seed_pixel(self):
        # 통상적으로 위내시경 영상의 roi는 우측에 존재합니다.
        # 따라서 우측으로 부터 (0,0)이 아닌 구역을 찾아줍니다.
        for i in range(self.w):
            if self.binary_image[self.h//2, self.w-(i+1)] == 255:
                return (self.h//2, self.w-(i+1))
            
    def define_roi(self):
        # roi 영역으로 유추되는 위치의 x1,x2,y1,y2를 구합니다.
        segmented_image = self.apply_region_growing()
        
        roi = np.where(segmented_image==255)
              
        y1 = roi[0].max()
        y2 = roi[0].min()
        x1 = roi[1].max()
        x2 = roi[1].min()

        return (x1, y1, x2, y2)

In [70]:
def video_load(path: str):
    """
        
    Args:
        ...

    Return:
        ...
    """
    cap = cv2.VideoCapture(path)   

    # 프레임 길이, 너비/높이, 초당 프레임 수 확인
    length = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    playback_time = length // fps
    playback_time_M = int(playback_time // 60)
    playback_time_S = int(playback_time - (playback_time_M * 60))
    
    print('프레임 길이: %d, 프레임 너비: %d, 프레임 높이: %d, 초당 프레임 수: %d' %(length, width, height, fps))
    print('비디어 재생시간: %s:%s' %(str(playback_time_M).zfill(2), str(playback_time_S).zfill(2)))
    print('=='*50)
    
    return cap, {'length':length, 'width': width, 'height': height, 'fps': fps, 
                    'playback_time': playback_time, 'playback_time_M': playback_time_M, 'playback_time_S': playback_time_S}

def crop_frame(frame: None, coord: list, new_width: int, new_height):
        """
            
        Args:
            ...

        Return:
            ...
        """
        cut_frame = frame[int(coord[1]):int(coord[3]), int(coord[0]):int(coord[2])]
               
        croped_frame = cv2.resize(src=cut_frame, 
                                  dsize=(new_width, new_height))
        
        croped_frame = croped_frame.reshape(new_width, new_height, 3)
        
        return frame, croped_frame


In [71]:
# video_paths = glob.glob(os.path.join('..', 'data', 'encode', '*.mp4'))

video_path = os.path.join('..', 'data', 'encode', '17754773_001.mp4')

print(video_path)

cap, cap_info = video_load(path=video_path)

count = 0

while cap.isOpened():
    ret, frame = cap.read()
    
    # 더이상 프레임이 존재하지 않을경우 종료
    if not ret:
        sys.exit()
    else:
        if count == cap_info['length'] // 60:
            # Region Growing 알고리즘을 적용합니다.
            exe = EGD_RegionGrow(frame)
            
            crop_roi = exe.define_roi()
            
            frame = cv2.rectangle(frame, (crop_roi[0],crop_roi[1]), (crop_roi[2],crop_roi[3]), (0,255,0), 2)
            
            # # 결과 이미지를 출력합니다.
            cv2.imshow('apply roi', frame)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
            
            cap.release()
                
        count += 1

..\data\encode\17754773_001.mp4
프레임 길이: 4009, 프레임 너비: 1280, 프레임 높이: 720, 초당 프레임 수: 60
비디어 재생시간: 01:06


c:\Users\user\anaconda3\envs\python_ksm\lib\site-packages\ipykernel_launcher.py:35: RuntimeWarning: overflow encountered in ubyte_scalars
